In [1]:
# Helper functions

def head_dict(d, n = 20, digits = 4):
    for key in sorted(d, key=d.get, reverse=True)[0:n]:
        print("%s: %s" % (key, round(d[key], digits)))

def normalize_frequencies(d):
    total = 0
    for key in d:
        total += d[key]
    for key in d:
        d[key] /= total_word_count
    return d

from collections import Counter

text = open('shakespeare.txt').read()
words = text.lower().replace(',', ' ').replace('.', ' ').split()

# Unigram
total_word_count = len(words)
word_frequencies = Counter(words)

unigram = normalize_frequencies(word_frequencies)

head_dict(unigram)

the: 0.0305
and: 0.0296
i: 0.0223
to: 0.0209
of: 0.0201
a: 0.0159
you: 0.0143
my: 0.0138
that: 0.0121
in: 0.012
is: 0.0105
not: 0.0093
for: 0.0091
with: 0.0088
it: 0.008
be: 0.0077
me: 0.0076
your: 0.0076
his: 0.0076
this: 0.007


In [2]:
# Bigram

bigram = {}

for i in range(0, len(words) - 1):
    key = (words[i], words[i+1])
    try:
        bigram[key] += 1
    except KeyError:
        bigram[key] = 1
normalize_frequencies(bigram)
head_dict(bigram)

('i', 'am'): 0.002
('of', 'the'): 0.0019
('in', 'the'): 0.0018
('i', 'have'): 0.0018
('i', 'will'): 0.0017
('to', 'the'): 0.0015
('my', 'lord'): 0.0015
('it', 'is'): 0.0011
('to', 'be'): 0.0011
('that', 'i'): 0.001
('and', 'the'): 0.0009
('i', 'do'): 0.0009
('and', 'i'): 0.0008
('of', 'my'): 0.0008
('you', 'are'): 0.0007
('is', 'the'): 0.0007
('i', 'would'): 0.0007
('the', 'king'): 0.0007
('you', 'have'): 0.0007
('he', 'is'): 0.0007


In [3]:
# Trigram

trigram = {}

for i in range(0, len(words) - 2):
    key = (words[i], words[i+1], words[i+2])
    try:
        trigram[key] += 1
    except KeyError:
        trigram[key] = 1
normalize_frequencies(trigram)
head_dict(trigram)

('so', 'long', 'as'): 0.0003
('works', 'of', 'william'): 0.0002
('of', 'william', 'shakespeare'): 0.0002
('complete', 'works', 'of'): 0.0002
('world', 'library', 'inc'): 0.0002
('the', 'complete', 'works'): 0.0002
('of', 'the', 'complete'): 0.0002
('by', 'project', 'gutenberg'): 0.0002
('service', 'that', 'charges'): 0.0002
('machine', 'readable', 'copies'): 0.0002
('any', 'service', 'that'): 0.0002
('use', 'only', 'and'): 0.0002
('library', 'inc', 'and'): 0.0002
('and', '(2)', 'are'): 0.0002
('includes', 'by', 'any'): 0.0002
('such', 'copies', '(1)'): 0.0002
('shakespeare', 'is', 'copyright'): 0.0002
('project', 'gutenberg', 'etext'): 0.0002
('copyright', '1990-1993', 'by'): 0.0002
('as', 'such', 'copies'): 0.0002


In [4]:
def shannon_ngram(words, ngram, length=10):
    if length == 0:
        return words
    
    n = len(list(ngram.keys())[0])
    next_word = ''
    max_p = 0
    
    for key in ngram:
        if words[-n+1:] == key[:n-1] and ngram[key] > max_p:
            next_word = key[-1]
            max_p = ngram[key]
            
    return shannon_ngram(words + (next_word,), ngram, length - 1)

print(shannon_ngram(('if',), bigram))
print(shannon_ngram(('if', 'you'), trigram))

print(shannon_ngram(('king',), bigram))
print(shannon_ngram(('king', 'henry'), trigram))

('if', 'you', 'are', 'not', 'to', 'the', 'king', 'henry', 'the', 'king', 'henry')
('if', 'you', 'will', 'not', 'be', 'so', 'bold', 'to', 'say', 'the', 'truth', 'of')
('king', 'henry', 'the', 'king', 'henry', 'the', 'king', 'henry', 'the', 'king', 'henry')
('king', 'henry', 'the', 'fifth', 'who', 'made', 'thee', 'faint', 'and', 'fly', 'like', 'chidden')


In [7]:
# Language model

train_data = words[0:round(len(words) * 0.8)]
validation_data = words[round(len(words) * 0.8)+1:]


def get_prob_func(frequency_table):
    n = len(list(frequency_table.keys())[0])
    assert n >= 2 # Doesn't work with unigrams
    
    def prob(words, conditional_words):
        if len(words) == n and len(conditional_words) == 0:
            total_count = 0
            target_count = 0
            for key in frequency_table:
                freq = frequency_table[key]
                if key == words:
                    target_count = freq
                total_count += freq
            return target_count / total_count
        
        elif len(words) == 1 and len(conditional_words) == n - 1:
            marginal_count = 0
            target_count= 0
            for key in frequency_table:
                if key[:n-1] == conditional_words:
                    marginal_count += frequency_table[key]
                    if key[-1] == words[0]:
                        target_count += frequency_table[key]
            return target_count / marginal_count
                    
        else:
            p = prob((words[-1],), words[-n:-1]) * prob(words[:-1], ())
            return p
    
    return prob


def get_trigram_language_model(words):
    trigram_table = {}
    for i in range(0, len(words) - 2):
        key = (words[i], words[i+1], words[i+2])
        try:
            trigram_table[key] += 1
        except KeyError:
            trigram_table[key] = 1

    prob = get_prob_func(trigram_table)
    
    def lm(sentence):
        return prob(sentence, ())
    
    return lm

lm = get_trigram_language_model(train_data)

print(lm(('consider', 'every', 'thing', 'that', 'grows')))

2.6377359784549723e-08


In [ ]:
# Perplexity

def get_perplexity_func(language_model):
    
    def perplexity(sentence):
        N = len(sentence)
        return language_model(sentence) ** (-1 / N)
    
    return perplexity

perp = get_perplexity_func(lm)
print(perp(tuple(words[:100])))

In [ ]:
# Smoothing
trigram_freq = {}

for i in range(0, len(words) - 2):
    key1 = (words[i], words[i+1])
    key2 = words[i+2]
    try:
        trigram_freq[key1][key2] += 1
    except KeyError:
        try: 
            trigram_freq[key1][key2] = 1
        except KeyError:
            trigram_freq[key1] = {}
            trigram_freq[key1][key2] = 1


In [ ]:
# Laplace
from copy import deepcopy

vocabulary = list(set(words))

trigram_freq_laplace = deepcopy(trigram_freq)

for word1 in vocabulary:
    for word2 in vocabulary:
        for word3 in vocabulary:
            key1 = (word1, word2)
            key2 = word3
            try:
                trigram_freq[key1][key2] += 1
            except KeyError:
                try: 
                    trigram_freq[key1][key2] = 1
                except KeyError:
                    trigram_freq[key1] = {}
                    trigram_freq[key1][key2] = 1